![imgs/hfb3.png](hfb3.png)
# HFB3 notebook demo

**Authors:** N. Dubray, J.-P. Ebran, P. Carpentier, M. Frosini, A. Zdeb, N. Pillet, J. Newsome, M. Verrière, G. Accorto, D. Regnier.  

This notebook presents some ways to use `HFB3` through its `Python` interface.

## Module initialization

In [ ]:
# disable OpenMP (HFB3 is meant to be used as a single-thread instance)
%env OMP_NUM_THREADS=1

# import the module
from hfb3 import *

## Load a previous result file

We start by loading a file containing the result of a previous run. This can be done with the class `DataTree`.

In [ ]:
filename = "42Ca_deformed_1x11.msg.gz"
dataTree = DataTree(filename)
print(dataTree)

Using a `DataTree` instance is the main way for `HFB3` to load and save information.  
Such an instance is similar to a Python `dict` object, as it stores data as a list of (key, value) couples.  
For example, the number of protons of the nuclear system described in this file can be obtained in the following way:

In [ ]:
 # get the Integer value associated with the key "system/nProt".
print(f'number of protons: {dataTree.getI("system/nProt")}')

## Construct a `State` instance

We construct a `State` instance from the previous `DataTree` instance. A `State` object represent a nuclear state. This can be done directly from the filename:

In [ ]:
state = State(filename) # alternative: state = State(DataTree(filename))
print(state)

One can see that this nuclear state correspond to 42Ca, and contains 364x364 rho and kappa matrices.
Such matrices are automatically converted to `numpy.array` instances:

In [ ]:
import numpy as np
print(f"norm of rho(NEUTRON): {np.linalg.norm(state.rho(NEUTRON), ord = 2)}")

## Calculate (num. int.) the multipole moments of the state

We can use the `Geometry` class to numerically calculate the mean value of the usual multipole moments for the previous nuclear state:

In [ ]:
print(Geometry(state))

## Calculate (ana. int.) the multipole moments of the state

The same can be done analytically by using a `MultipoleOperators` instance:

In [ ]:
print(MultipoleOperators(state))

## Calculate energy contributions

We calculate and print the energy contribution associated with each part of the effective nucleon-nucleon interaction D1S for the previous nuclear state:

In [ ]:
interaction = Interaction("D1S", state)
interaction.calcEnergies()
print(interaction)

One can also use an alternate display for such energy contributions:

In [ ]:
print(interaction.getNiceInfo())

## Calculate and plot local the local 1-body density (using `matplotlib`)

We calculate and plot the local one-body density in the $(r_\perp, z)$ plane. This is done using an instance of the `Discrete` class.

In [ ]:
def plot_matplotlib(_state, zmin, zmax, xmin, xmax):
    discrete = Discrete(_state.basis, Mesh.regular(xmin, 0, zmin, xmax, 0, zmax, 101, 1, 201))
    denst = discrete.getLocalXZ(_state.rho(NEUTRON) + _state.rho(PROTON), True)
  
    import numpy as np
    from matplotlib.pyplot import cm
    import matplotlib.pyplot as plt
    from mpl_toolkits.axes_grid1 import make_axes_locatable

    fig = plt.figure(1, figsize=(12, 12))
    ax = plt.gca()
    im = ax.imshow(np.flip(denst, axis = 0), cmap = cm.inferno, extent = [zmin, zmax, xmin, xmax])
    plt.xlabel(r'$z$ [fm]')
    plt.ylabel(r'$r$ [fm]')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    plt.colorbar(im, cax=cax)
    plt.ylabel(r'$\rho_{tot}$ [fm$^{-3}$]')
    plt.show()
    
plot_matplotlib(state, -10, 10, 0, 6)

## Calculate and plot local the local 1-body density (using `bokeh`)

We can do the same plot with the `bokeh` Python module:

In [ ]:
def plot_bokeh(_state, zmin, zmax, xmin, xmax):
    discrete = Discrete(_state.basis, Mesh.regular(xmin, 0, zmin, xmax, 0, zmax, 101, 1, 201))
    denst = discrete.getLocalXZ(_state.rho(NEUTRON) + _state.rho(PROTON), True)
  
    from bokeh.io import output_notebook
    output_notebook()
    from bokeh.plotting import figure, show
    from bokeh.models import ColorBar, LinearColorMapper
    p = figure(tools = "pan, reset, save, wheel_zoom", height = 300, width = 800,
               active_drag = "pan", active_scroll = "wheel_zoom", match_aspect = True,
               x_axis_label = 'z [fm]', y_axis_label = 'r [fm]')
    color = LinearColorMapper(palette = "Inferno256")
    p.image(image = [denst], x = zmin, y = xmin, dw = zmax - zmin, dh = xmax - xmin, color_mapper = color)
    color_bar = ColorBar(color_mapper = color, label_standoff = 10, location = (0,0), width = 10)
    p.add_layout(color_bar, 'right')
    show(p)

plot_bokeh(state, -10, 10, 0, 6)

## Perform a 1ct Constrained HFB calculation

Here, we calculate the HFB nuclear state minimizing the total binding energy under the action of a constraint on the quadrupole mass moment, using a fixed 1-center HO basis. The solver uses the Broyden mixing method. The basis parameters and the initial HFB state are taken from a previous result stored in file (`42Ca_deformed_1x11.msg.gz`). A constraint on the center of mass of the system (`q10t`) is automatically set with the value 0.0.

In [ ]:
%env OMP_NUM_THREADS=1
dataTree = DataTree("42Ca_deformed_1x11.msg.gz")
dataTree.setD("constraints/q20t", 15.0)
solverHFB = SolverHFBBroyden(dataTree)
solverHFB.lambdaMax = 1e-03
solverHFB.init()
while(solverHFB.nextIter()): pass
print(solverHFB.interaction.getNiceInfo())
print(MultipoleOperators(solverHFB.state).getNiceInfo())

The final total HFB energy should be -361.695 MeV. In the "Deformations" table, the constrained values are printed in blue.

## Perform a 2ct Constrained HFB calculation (with live visualization using `bokeh`)

We now calculate a similar HFB nuclear state, using a 2-center HO basis. A separate bokeh server can be launched to generate a live visualization of the nuclear state during HFB convergence.

In [ ]:
%env OMP_NUM_THREADS=1

# launch a bokeh server before this (`bin/bokehserver.py`), and open the proposed page in your browser.
cvar.useBokeh = True
Plot.clear()

dataTree = DataTree("42Ca_deformed_2x9.msg.gz")
#print(MultipoleOperators(BogoliubovState(dataTree)))
dataTree.setD("constraints/q20t", 12.0)
solverHFBBroyden = SolverHFBBroyden(dataTree)
solverHFBBroyden.plotDensities = True
solverHFBBroyden.init()
print(solverHFBBroyden)
while(solverHFBBroyden.nextIter()): pass
print(solverHFBBroyden.interaction.getNiceInfo())
print(MultipoleOperators(solverHFBBroyden.state).getNiceInfo())

cvar.useBokeh = False

The final total HFB energy should be -362.181 MeV.

## Perform a 2ct Constrained HFB calculation (with `SolverHFBGrad`)

For this HFB calculation, we use a solver implementing the Gradiend method.

In [ ]:
%env OMP_NUM_THREADS=1
dataTree = DataTree("42Ca_deformed_2x9.msg.gz")
dataTree.setD("constraints/q20t", 12.0)
dataTree.setI("solver/gradient/maxIter", 100)
dataTree.setD("solver/gradient/cvgTarget", 1e-6)
solverHFBGradient = SolverHFBGradient(dataTree)
solverHFBGradient.init()
print(solverHFBGradient)
while(solverHFBGradient.nextIter()): pass
print(solverHFBGradient.interaction.getNiceInfo())
print(MultipoleOperators(solverHFBGradient.state).getNiceInfo())

The final total HFB energy should be -362.181 MeV, and the multipole moments should be close from the ones found with the Broyden-mixing solver.

## Compare the multipole moments between solvers

One can compare the total multipole moments for both solvers with a numerical integration:

In [ ]:
print(Geometry(solverHFBGradient.state))
print(Geometry(solverHFBBroyden.state))

## Plot the local 1-body densities

We can plot the local 1-body nuclear densities for each solver:

In [ ]:
plot_bokeh(solverHFBGradient.state, -10, 10, -6, 6)
plot_bokeh(solverHFBBroyden.state, -10, 10, -6, 6)